In [1]:
import numpy as np


In [2]:
# 初始化参数
num_servers = 10  # 服务器数量
num_contents = 20  # 内容数量
R = np.random.rand(num_servers)  # 服务器稳定性评分
I = np.random.rand(num_contents)  # 内容重要性评分
connections = [np.random.choice(num_servers, 3, replace=False) for _ in range(num_servers)]  # 每个服务器的连接

# 初始化策略概率
P = np.random.rand(num_servers, num_contents)  # 每个服务器缓存内容的概率

In [3]:
max_iter = 100
tolerance = 1e-3

In [4]:
def global_objective(P, R, I):
    G = 0
    for i in range(num_servers):
        for k in range(num_contents):
            G += P[i, k] * I[k] * R[i]
    return G

In [5]:
# 复制动态方程更新策略
def update_strategy(P, R, I, connections):
    for i in range(num_servers):
        for k in range(num_contents):
            neighbors = connections[i]
            cache_benefit = I[k] * R[i]  # 缓存带来的收益
            offload_benefit = np.mean([P[j, k] * I[k] * R[j] for j in neighbors])  # 转交邻居的收益
            avg_benefit = P[i, k] * cache_benefit + (1 - P[i, k]) * offload_benefit
            P[i, k] += P[i, k] * (cache_benefit - avg_benefit)
    return np.clip(P, 0, 1)  # 限制策略概率在 [0, 1]

In [12]:
for iter in range(max_iter):
    prev_G = global_objective(P, R, I)
    P = update_strategy(P, R, I, connections)
    curr_G = global_objective(P, R, I)
    if abs(curr_G - prev_G) < tolerance:
        break

print("Final global objective:", curr_G)

Final global objective: 50.08014054435253
